<a href="https://colab.research.google.com/github/hoangtung386/Kaggle_notebook/blob/main/Segment_RSNA_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rsna_2023_abdominal_trauma_detection_path = kagglehub.competition_download('rsna-2023-abdominal-trauma-detection')

print('Data source import complete.')


# Install packages

In [ ]:
!pip install -q monai wandb nibabel torchsummary
!pip install -q protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 44.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's depend

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn.metrics import confusion_matrix
from monai.transforms import (Compose, LoadImaged, EnsureChannelFirstd, Spacingd, ScaleIntensityRanged,
                              Orientationd, CropForegroundd, SpatialPadd, RandAffined, ToTensord)
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.data import CacheDataset, list_data_collate
from monai.utils import set_determinism
from tqdm import tqdm
from scipy.spatial.distance import directed_hausdorff
import wandb
from datetime import datetime
import nibabel as nib

<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-11-20 19:07:08.065420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763665628.227135      89 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763665628.276872      89 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it w

# W&B Authentication

In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
use_wandb = True  # Đặt thành False nếu không muốn dùng wandb

In [ ]:
if use_wandb:
    try:
        wandb_api = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api)
        print("W&B authentication successful!")
    except Exception as e:
        print(f"W&B authentication failed: {e}")
        print("Continuing without W&B logging")
        use_wandb = False

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hoangtung386 (levuhoangtung) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B authentication successful!


In [ ]:
# Set seed
random_seed = 42
set_determinism(seed=random_seed)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# I . Data preprocessing

## 1. Paths và Config

In [ ]:
base_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection/'
output_dir = '/kaggle/working'
os.makedirs(output_dir, exist_ok=True)

CONFIG = {
    'seed': random_seed,
    'train_split': 0.8,
    'batch_size': 32,
    'num_epochs': 60,
    'learning_rate': 1e-3,
    'spatial_size': (256, 256),
    'init_features': 32,
    'num_classes': 6,
    'cache_rate': 0,
    'num_workers': 2,
}

## 2. Load Data and Extract 2D Slices

In [ ]:
series_meta = pd.read_csv(os.path.join(base_path, 'train_series_meta.csv'))
labels = pd.read_csv(os.path.join(base_path, 'train_2024.csv'))

seg_path = os.path.join(base_path, 'segmentations')
seg_files = [f for f in os.listdir(seg_path) if f.endswith('.nii')]

In [ ]:
print("Extracting 2D slices from 3D volumes")
data_list = []

for seg_file in tqdm(seg_files, desc="Processing volumes"):
    series_id = int(seg_file.split('.')[0])
    patient_row = series_meta[series_meta['series_id'] == series_id]

    if not patient_row.empty:
        patient_id = int(patient_row['patient_id'].values[0])
        image_dir = os.path.join(base_path, f'train_images/{patient_id}/{series_id}')
        seg_file_path = os.path.join(seg_path, seg_file)

        # Load segmentation to get number of slices
        try:
            seg_nii = nib.load(seg_file_path)
            seg_data = seg_nii.get_fdata()

            # Get slices that contain organs (not all background)
            num_slices = seg_data.shape[2]
            for slice_idx in range(num_slices):
                slice_seg = seg_data[:, :, slice_idx]
                # Only include slices with at least some organ segmentation
                if np.sum(slice_seg > 0) > 100:  # At least 100 pixels with organs
                    data_list.append({
                        'image': image_dir,
                        'seg': seg_file_path,
                        'slice_idx': slice_idx
                    })
        except Exception as e:
            print(f"Error processing {seg_file}: {e}")
            continue

print(f"Total 2D slices extracted: {len(data_list)}")

Extracting 2D slices from 3D volumes


Processing volumes:  18%|█▊        | 37/206 [01:10<04:51,  1.73s/it]

In [ ]:
train_list, val_list = train_test_split(
    data_list,
    test_size=1-CONFIG['train_split'],
    random_state=CONFIG['seed'],
    shuffle=True
)
print(f"Train slices: {len(train_list)}, Val slices: {len(val_list)}")

## 3. Custom 2D Dataset

In [ ]:
class RSNA2DDataset(Dataset):
    def __init__(self, data_list, transforms=None, spatial_size=(256, 256)):
        self.data_list = data_list
        self.transforms = transforms
        self.spatial_size = spatial_size

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data_dict = self.data_list[idx]
        image_dir = data_dict['image']
        seg_path = data_dict['seg']
        slice_idx = data_dict['slice_idx']

        # Load 3D volumes
        from monai.transforms import LoadImage
        loader = LoadImage(image_only=True)

        # Load image (DICOM series)
        image_3d = loader(image_dir)
        # Load segmentation
        seg_3d = loader(seg_path)

        # Convert to numpy if torch tensor
        if isinstance(image_3d, torch.Tensor):
            image_3d = image_3d.numpy()
        if isinstance(seg_3d, torch.Tensor):
            seg_3d = seg_3d.numpy()

        # Extract 2D slice
        if len(image_3d.shape) == 4:
            image_2d = image_3d[0, :, :, slice_idx]
        else:
            image_2d = image_3d[:, :, slice_idx]

        if len(seg_3d.shape) == 4:
            seg_2d = seg_3d[0, :, :, slice_idx]
        else:
            seg_2d = seg_3d[:, :, slice_idx]

        # Ensure numpy array
        image_2d = np.array(image_2d, dtype=np.float32)
        seg_2d = np.array(seg_2d, dtype=np.float32)

        # Ensure 2D shape (H, W)
        if len(image_2d.shape) == 3:
            image_2d = image_2d[0]
        if len(seg_2d.shape) == 3:
            seg_2d = seg_2d[0]

        # Apply transforms
        if self.transforms:
            # Intensity scaling
            image_2d = np.clip(image_2d, -125, 275)
            image_2d = (image_2d + 125) / 400.0  # Normalize to [0, 1]

            # Crop foreground
            mask = image_2d > 0.1
            if mask.sum() > 0:
                coords = np.argwhere(mask)
                y_min, x_min = coords.min(axis=0)
                y_max, x_max = coords.max(axis=0)

                # Add padding
                pad = 10
                y_min = max(0, y_min - pad)
                x_min = max(0, x_min - pad)
                y_max = min(image_2d.shape[0], y_max + pad)
                x_max = min(image_2d.shape[1], x_max + pad)

                image_2d = image_2d[y_min:y_max, x_min:x_max]
                seg_2d = seg_2d[y_min:y_max, x_min:x_max]

            # Resize to fixed size
            from skimage.transform import resize
            image_2d = resize(image_2d, self.spatial_size, order=1, preserve_range=True)
            seg_2d = resize(seg_2d, self.spatial_size, order=0, preserve_range=True)

        # Add channel dimension (C, H, W)
        image_2d = image_2d[np.newaxis, ...].astype(np.float32)
        seg_2d = seg_2d[np.newaxis, ...].astype(np.int64)

        # Convert to torch tensors
        image_2d = torch.from_numpy(image_2d)
        seg_2d = torch.from_numpy(seg_2d)

        return {'image': image_2d, 'seg': seg_2d}

## 4. Data processing

In [ ]:
train_ds = RSNA2DDataset(train_list, transforms=True, spatial_size=CONFIG['spatial_size'])
val_ds = RSNA2DDataset(val_list, transforms=True, spatial_size=CONFIG['spatial_size'])

## 5. DataLoader

In [ ]:
train_loader = DataLoader(
    train_ds,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=1,
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

# II. Unet 2D Architecture

## 1. Initialize the architecture

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

In [ ]:
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

In [ ]:
class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # Handle size mismatch
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = nn.functional.pad(x1, [diffX // 2, diffX - diffX // 2,
                                    diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [ ]:
class UNet2D(nn.Module):
    def __init__(self, in_channels=1, out_channels=6, init_features=32):
        super().__init__()
        features = init_features

        self.inc = DoubleConv(in_channels, features)
        self.down1 = Down(features, features * 2)
        self.down2 = Down(features * 2, features * 4)
        self.down3 = Down(features * 4, features * 8)
        self.down4 = Down(features * 8, features * 16)

        self.up1 = Up(features * 16, features * 8)
        self.up2 = Up(features * 8, features * 4)
        self.up3 = Up(features * 4, features * 2)
        self.up4 = Up(features * 2, features)

        self.outc = nn.Conv2d(features, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        logits = self.outc(x)
        return logits

## 2. Initialize model

In [ ]:
from torchsummary import summary

model = UNet2D(
    in_channels=1,
    out_channels=CONFIG['num_classes'],
    init_features=CONFIG['init_features']
).to(device)

try:
    summary(model, (1, CONFIG['spatial_size'][0], CONFIG['spatial_size'][1]))
except:
    print("Model summary not available, but model created successfully")

## 3. Trainer Class

In [ ]:
class UNetTrainer:
    def __init__(self, model, train_loader, val_loader, config, device,
                 checkpoint_dir='/kaggle/working', use_wandb=True):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.checkpoint_dir = checkpoint_dir
        self.use_wandb = use_wandb

        self.device = device
        self.model.to(self.device)

        # Loss and optimizer
        self.criterion = DiceLoss(to_onehot_y=True, softmax=True)
        self.optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='max', patience=5, factor=0.5, verbose=True
        )
        self.dice_metric = DiceMetric(include_background=False, reduction='mean')

        # Training state
        self.start_epoch = 0
        self.best_dice = 0.0
        self.history = []
        self.wandb_run_id = None

        # Paths
        self.checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint_2d.pth')
        self.best_model_path = os.path.join(checkpoint_dir, 'best_model_2d.pth')
        self.history_csv_path = os.path.join(checkpoint_dir, 'training_history_2d.csv')

    def save_checkpoint(self, epoch, val_dice, is_best=False):
        """Save checkpoint"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_dice': self.best_dice,
            'history': self.history,
            'config': self.config,
            'wandb_run_id': self.wandb_run_id
        }

        torch.save(checkpoint, self.checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch}")

        if is_best:
            torch.save(checkpoint, self.best_model_path)
            print(f"Best model saved! Dice: {val_dice:.4f}")

    def load_checkpoint(self):
        """Load checkpoint if exists"""
        if os.path.exists(self.checkpoint_path):
            print(f"Loading checkpoint from {self.checkpoint_path}")
            checkpoint = torch.load(self.checkpoint_path, map_location=self.device)

            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            self.start_epoch = checkpoint['epoch'] + 1
            self.best_dice = checkpoint['best_dice']
            self.history = checkpoint['history']

            if 'wandb_run_id' in checkpoint:
                self.wandb_run_id = checkpoint['wandb_run_id']

            print(f"Resumed from epoch {self.start_epoch}, best dice: {self.best_dice:.4f}")
            return True
        return False

    def save_history_csv(self):
        """Save training history to CSV"""
        if self.history:
            df = pd.DataFrame(self.history)
            df.to_csv(self.history_csv_path, index=False)

    def train_epoch(self, epoch):
        """Train one epoch"""
        self.model.train()
        total_loss = 0

        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch} [Train]")
        for batch in pbar:
            images = batch['image'].to(self.device)
            segs = batch['seg'].to(self.device)

            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, segs)
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / len(self.train_loader)
        return avg_loss

    def validate(self, epoch):
        """Validate"""
        self.model.eval()
        self.dice_metric.reset()

        with torch.no_grad():
            pbar = tqdm(self.val_loader, desc=f"Epoch {epoch} [Val]")
            for batch in pbar:
                images = batch['image'].to(self.device)
                segs = batch['seg'].to(self.device)

                outputs = self.model(images)
                self.dice_metric(y_pred=outputs, y=segs)

        val_dice = self.dice_metric.aggregate().item()
        return val_dice

    def train(self, num_epochs=None):
        """Main training loop"""
        if num_epochs is None:
            num_epochs = self.config['num_epochs']

        resumed = self.load_checkpoint()

        if self.use_wandb:
            if resumed and self.wandb_run_id:
                wandb.init(
                    project="my-2D-Unet-segment-RSNA",
                    config=self.config,
                    resume="allow",
                    id=self.wandb_run_id,
                    name=f"unet2d_resumed_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                )
                print(f"Resumed W&B run: {self.wandb_run_id}")
            else:
                run_name = f"unet2d_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                wandb.init(
                    project="my-2D-Unet-segment-RSNA",
                    config=self.config,
                    name=run_name,
                    tags=["2d-unet", "organ-segmentation", "rsna"]
                )
                self.wandb_run_id = wandb.run.id
                print(f"Created new W&B run: {self.wandb_run_id}")

            wandb.watch(self.model, log='all', log_freq=100)

        print(f"\nStarting training from epoch {self.start_epoch} to {num_epochs}")
        print(f"Device: {self.device}")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()) / 1e6:.2f}M")

        for epoch in range(self.start_epoch, num_epochs):
            train_loss = self.train_epoch(epoch + 1)
            val_dice = self.validate(epoch + 1)
            self.scheduler.step(val_dice)

            current_lr = self.optimizer.param_groups[0]['lr']

            metrics = {
                'epoch': epoch + 1,
                'train_loss': train_loss,
                'val_dice': val_dice,
                'learning_rate': current_lr,
                'best_dice': self.best_dice
            }

            self.history.append(metrics)

            print(f"\nEpoch {epoch+1}/{num_epochs}:")
            print(f"  Train Loss: {train_loss:.4f}")
            print(f"  Val Dice: {val_dice:.4f}")
            print(f"  LR: {current_lr:.6f}")
            print(f"  Best Dice: {self.best_dice:.4f}")

            if self.use_wandb:
                wandb.log(metrics)

            is_best = val_dice > self.best_dice
            if is_best:
                self.best_dice = val_dice

            self.save_checkpoint(epoch, val_dice, is_best)
            self.save_history_csv()

        print(f"\nTraining complete! Best Dice: {self.best_dice:.4f}")

        if self.use_wandb:
            wandb.finish()

## 4. Create trainer

In [ ]:
trainer = UNetTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    config=CONFIG,
    device=device,
    checkpoint_dir=output_dir,
    use_wandb=use_wandb
)

# III. Start training

In [ ]:
# Clean memory
import gc
gc.collect()
torch.cuda.empty_cache()

trainer.train(num_epochs=CONFIG['num_epochs'])

# IV. Load Best Model for Inference

In [ ]:
best_checkpoint = torch.load(trainer.best_model_path, map_location=device)
model.load_state_dict(best_checkpoint['model_state_dict'])
print(f"Best model loaded from epoch {best_checkpoint['epoch']} with Dice: {best_checkpoint['best_dice']:.4f}")

# V. Evaluation & Visualization

In [ ]:
class SegmentationEvaluator:
    def __init__(self, model, val_loader, device, num_classes=6):
        self.model = model
        self.val_loader = val_loader
        self.device = device
        self.num_classes = num_classes
        self.class_names = ['Background', 'Liver', 'Spleen', 'Kidney_L', 'Kidney_R', 'Bowel']

    def compute_metrics(self):
        """Compute comprehensive metrics"""
        self.model.eval()

        class_metrics = {cls: {
            'dice': [], 'iou': [], 'precision': [],
            'recall': [], 'specificity': []
        } for cls in range(1, self.num_classes)}

        print("Computing metrics on validation set...")
        with torch.no_grad():
            for batch in tqdm(self.val_loader):
                images = batch['image'].to(self.device)
                labels = batch['seg'].to(self.device)

                outputs = self.model(images)
                preds = torch.argmax(outputs, dim=1)

                pred_np = preds.cpu().numpy()
                label_np = labels.cpu().numpy()

                for cls in range(1, self.num_classes):
                    pred_cls = (pred_np == cls).astype(np.float32)
                    label_cls = (label_np == cls).astype(np.float32)

                    if label_cls.sum() == 0:
                        continue

                    intersection = (pred_cls * label_cls).sum()
                    dice = (2 * intersection) / (pred_cls.sum() + label_cls.sum() + 1e-8)
                    class_metrics[cls]['dice'].append(dice)

                    union = pred_cls.sum() + label_cls.sum() - intersection
                    iou = intersection / (union + 1e-8)
                    class_metrics[cls]['iou'].append(iou)

                    tp = intersection
                    fp = pred_cls.sum() - intersection
                    fn = label_cls.sum() - intersection
                    tn = ((pred_cls == 0) & (label_cls == 0)).sum()

                    precision = tp / (tp + fp + 1e-8)
                    recall = tp / (tp + fn + 1e-8)
                    specificity = tn / (tn + fp + 1e-8)

                    class_metrics[cls]['precision'].append(precision)
                    class_metrics[cls]['recall'].append(recall)
                    class_metrics[cls]['specificity'].append(specificity)

        results = {}
        for cls in range(1, self.num_classes):
            results[self.class_names[cls]] = {
                metric: np.mean(values) if values else 0.0
                for metric, values in class_metrics[cls].items()
            }

        return results

    def plot_metrics(self, results):
        """Plot metrics comparison"""
        metrics_to_plot = ['dice', 'iou', 'precision', 'recall', 'specificity']

        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.flatten()

        for idx, metric in enumerate(metrics_to_plot):
            classes = list(results.keys())
            values = [results[cls][metric] for cls in classes]

            axes[idx].bar(classes, values, color='steelblue', alpha=0.8)
            axes[idx].set_title(f'{metric.upper()}', fontsize=14, fontweight='bold')
            axes[idx].set_ylabel('Score', fontsize=12)
            axes[idx].set_ylim([0, 1])
            axes[idx].grid(axis='y', alpha=0.3)
            axes[idx].tick_params(axis='x', rotation=45)

            for i, v in enumerate(values):
                axes[idx].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10)

        axes[5].axis('off')

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'metrics_comparison_2d.png'), dpi=150, bbox_inches='tight')
        plt.show()
        print(f"Metrics plot saved to {output_dir}/metrics_comparison_2d.png")

    def visualize_predictions(self, num_samples=5):
        """Visualize overlay masks for 2D slices"""
        self.model.eval()

        colors = ['black', 'red', 'green', 'blue', 'yellow', 'purple']
        cmap = ListedColormap(colors)

        samples_shown = 0

        with torch.no_grad():
            for batch in self.val_loader:
                if samples_shown >= num_samples:
                    break

                images = batch['image'].to(self.device)
                labels = batch['seg'].to(self.device)

                outputs = self.model(images)
                preds = torch.argmax(outputs, dim=1)

                img_np = images[0, 0].cpu().numpy()
                pred_np = preds[0].cpu().numpy()
                label_np = labels[0, 0].cpu().numpy()

                fig, axes = plt.subplots(1, 3, figsize=(15, 5))

                # Original image
                axes[0].imshow(img_np, cmap='gray')
                axes[0].set_title('CT Image', fontsize=14, fontweight='bold')
                axes[0].axis('off')

                # Ground truth
                axes[1].imshow(img_np, cmap='gray')
                axes[1].imshow(label_np, cmap=cmap, alpha=0.5, vmin=0, vmax=5)
                axes[1].set_title('Ground Truth', fontsize=14, fontweight='bold')
                axes[1].axis('off')

                # Prediction
                axes[2].imshow(img_np, cmap='gray')
                axes[2].imshow(pred_np, cmap=cmap, alpha=0.5, vmin=0, vmax=5)
                axes[2].set_title('Prediction', fontsize=14, fontweight='bold')
                axes[2].axis('off')

                # Add legend
                from matplotlib.patches import Patch
                legend_elements = [Patch(facecolor=colors[i], label=self.class_names[i])
                                 for i in range(1, len(colors))]
                fig.legend(handles=legend_elements, loc='lower center', ncol=5, fontsize=12)

                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, f'overlay_2d_sample_{samples_shown+1}.png'),
                           dpi=150, bbox_inches='tight')
                plt.show()

                samples_shown += 1

        print(f"Overlay visualizations saved to {output_dir}/overlay_2d_sample_*.png")

    def plot_per_class_comparison(self, num_samples=3):
        """Plot detailed per-class segmentation comparison"""
        self.model.eval()

        colors = ['black', 'red', 'green', 'blue', 'yellow', 'purple']

        samples_shown = 0

        with torch.no_grad():
            for batch in self.val_loader:
                if samples_shown >= num_samples:
                    break

                images = batch['image'].to(self.device)
                labels = batch['seg'].to(self.device)

                outputs = self.model(images)
                preds = torch.argmax(outputs, dim=1)

                img_np = images[0, 0].cpu().numpy()
                pred_np = preds[0].cpu().numpy()
                label_np = labels[0, 0].cpu().numpy()

                # Create figure with per-organ comparison
                fig, axes = plt.subplots(2, self.num_classes, figsize=(20, 8))

                for cls_idx in range(self.num_classes):
                    # Ground truth for this class
                    gt_mask = (label_np == cls_idx).astype(float)
                    axes[0, cls_idx].imshow(img_np, cmap='gray')
                    axes[0, cls_idx].imshow(gt_mask, cmap='Reds', alpha=0.5, vmin=0, vmax=1)
                    axes[0, cls_idx].set_title(f'{self.class_names[cls_idx]}\nGround Truth',
                                               fontsize=10, fontweight='bold')
                    axes[0, cls_idx].axis('off')

                    # Prediction for this class
                    pred_mask = (pred_np == cls_idx).astype(float)
                    axes[1, cls_idx].imshow(img_np, cmap='gray')
                    axes[1, cls_idx].imshow(pred_mask, cmap='Blues', alpha=0.5, vmin=0, vmax=1)
                    axes[1, cls_idx].set_title(f'{self.class_names[cls_idx]}\nPrediction',
                                               fontsize=10, fontweight='bold')
                    axes[1, cls_idx].axis('off')

                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, f'per_class_comparison_{samples_shown+1}.png'),
                           dpi=150, bbox_inches='tight')
                plt.show()

                samples_shown += 1

        print(f"Per-class comparison saved to {output_dir}/per_class_comparison_*.png")

    def plot_confusion_analysis(self):
        """Plot confusion matrix for segmentation classes"""
        self.model.eval()

        all_preds = []
        all_labels = []

        print("Computing confusion matrix...")
        with torch.no_grad():
            for batch in tqdm(self.val_loader):
                images = batch['image'].to(self.device)
                labels = batch['seg'].to(self.device)

                outputs = self.model(images)
                preds = torch.argmax(outputs, dim=1)

                all_preds.append(preds.cpu().numpy().flatten())
                all_labels.append(labels.cpu().numpy().flatten())

        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        # Compute confusion matrix
        cm = confusion_matrix(all_labels, all_preds, labels=range(self.num_classes))

        # Normalize
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # Plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

        # Raw counts
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=self.class_names, yticklabels=self.class_names,
                    ax=ax1, cbar_kws={'label': 'Count'})
        ax1.set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
        ax1.set_ylabel('True Label', fontsize=12)
        ax1.set_xlabel('Predicted Label', fontsize=12)

        # Normalized
        sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Greens',
                    xticklabels=self.class_names, yticklabels=self.class_names,
                    ax=ax2, cbar_kws={'label': 'Proportion'})
        ax2.set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
        ax2.set_ylabel('True Label', fontsize=12)
        ax2.set_xlabel('Predicted Label', fontsize=12)

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'confusion_matrix_2d.png'), dpi=150, bbox_inches='tight')
        plt.show()
        print(f"Confusion matrix saved to {output_dir}/confusion_matrix_2d.png")

    def create_summary_report(self, results):
        """Create text summary report"""
        report_path = os.path.join(output_dir, 'evaluation_report_2d.txt')

        with open(report_path, 'w') as f:
            f.write("="*60 + "\n")
            f.write("2D SEGMENTATION EVALUATION REPORT\n")
            f.write("="*60 + "\n\n")

            for cls_name, metrics in results.items():
                f.write(f"\n{cls_name}:\n")
                f.write("-" * 40 + "\n")
                f.write(f"  Dice Coefficient:  {metrics['dice']:.4f}\n")
                f.write(f"  IoU:              {metrics['iou']:.4f}\n")
                f.write(f"  Precision:        {metrics['precision']:.4f}\n")
                f.write(f"  Recall:           {metrics['recall']:.4f}\n")
                f.write(f"  Specificity:      {metrics['specificity']:.4f}\n")

            f.write("\n" + "-"*60 + "\n")
            f.write("OVERALL AVERAGES:\n")
            f.write("-"*60 + "\n")

            for metric in ['dice', 'iou', 'precision', 'recall', 'specificity']:
                avg = np.mean([results[cls][metric] for cls in results.keys()])
                f.write(f"  Average {metric.upper()}: {avg:.4f}\n")

        print(f"\nEvaluation report saved to {report_path}")

        with open(report_path, 'r') as f:
            print(f.read())

In [ ]:
evaluator = SegmentationEvaluator(model, val_loader, device, num_classes=CONFIG['num_classes'])

## 1. Compute all metrics

In [ ]:
print("Computing metrics...")
results = evaluator.compute_metrics()

## 2. Plot metrics comparison

In [ ]:
print("Plotting metrics comparison")
evaluator.plot_metrics(results)

## 3. Visualize overlay masks

In [ ]:
print("Creating overlay visualizations...")
evaluator.visualize_predictions(num_samples=5)

## 4. Per-class comparison

In [ ]:
print("Creating per-class comparison...")
evaluator.plot_per_class_comparison(num_samples=3)

## 5. Confusion matrix analysis

In [ ]:
print("Creating confusion matrix...")
evaluator.plot_confusion_analysis()

## 6. Create summary report

In [ ]:
evaluator.create_summary_report(results)